# AIX360

In [1]:
import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from aix360.algorithms.contrastive import CEMExplainer, KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical

C:\Users\gerar\anaconda3\envs\AIX360\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\gerar\anaconda3\envs\AIX360\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\gerar\anaconda3\envs\AIX360\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\gerar\anaconda3\envs\AIX360\lib\site-packages\tensorflow\pyth

In [30]:
data = pd.read_csv('./archive/german_credit_data.csv')
data_copy = data.copy()
data_copy.fillna(value="unknown", inplace=True)

labels = data_copy.columns

def normalize(df):
    result = df.copy()
    max_value = df.max()
    min_value = df.min()
    result = (df - min_value) / (max_value - min_value)
    return result

for col in labels:
    if is_string_dtype(data_copy[col]):
        if col == 'Risk':
            data_copy[col] = pd.factorize(data_copy[col])[0] #PD Factorize da un valor numerico para cada posible valor de la columna
                                                             # por lo que al encontrar primero un good, good = 0 y bad = 1   
            continue
        
        data_copy = pd.concat([data_copy, pd.get_dummies(data_copy[col], prefix=col)], axis=1)
        data_copy.drop(col, axis=1, inplace=True)
    else:
        data_copy[col] = normalize(data_copy[col])

data_copy = data_copy[[c for c in data_copy if c not in ["Risk"]] + ["Risk"]]
display(data_copy)
data_copy.to_excel("data_processed_aix360.xlsx")

,Id,Age,Job,Credit amount,Duration,Sex_female,Sex_male,Housing_free,Housing_own,Housing_rent,...,Checking account_unknown,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Risk
0,0.000000,0.857143,0.666667,0.050567,0.029412,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,0.001001,0.053571,0.666667,0.313690,0.647059,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
2,0.002002,0.535714,0.333333,0.101574,0.117647,0,1,0,1,0,...,1,0,0,0,1,0,0,0,0,0
3,0.003003,0.464286,0.666667,0.419941,0.558824,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0.004004,0.607143,0.666667,0.254209,0.294118,0,1,1,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.995996,0.214286,0.333333,0.081765,0.117647,1,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
996,0.996997,0.375000,1.000000,0.198470,0.382353,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
997,0.997998,0.339286,0.666667,0.030483,0.117647,0,1,0,1,0,...,1,0,0,0,0,0,1,0,0,0
998,0.998999,0.071429,0.666667,0.087763,0.602941,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,1


In [31]:
# Preparation of data for training
X = data_copy.drop("Risk", axis=1) # Datos de entrada
Y = data_copy['Risk'] # Datos de salida 

# Separate into x_train, x_test, y_train y y_set
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

In [35]:
#Creates the model that the classifier will use in a future
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=27, activation='relu')) # Add a 
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_model()

params = {"epochs": 50}
model.fit(x_train, y_train, epochs=params["epochs"], validation_data=(x_test, y_test))
y_pred = model.predict_classes(x_test.values)

print("Precision score on validation data is {}".format(precision_score(y_test, y_pred, average="weighted")))

Train on 750 samples, validate on 250 samples
Epoch 1/50
750/750 [==============================] - 0s 466us/step - loss: 0.6923 - accuracy: 0.5480 - val_loss: 0.6731 - val_accuracy: 0.6200
Epoch 2/50
750/750 [==============================] - 0s 69us/step - loss: 0.6409 - accuracy: 0.6880 - val_loss: 0.6520 - val_accuracy: 0.6520
Epoch 3/50
750/750 [==============================] - 0s 71us/step - loss: 0.6131 - accuracy: 0.7013 - val_loss: 0.6408 - val_accuracy: 0.6600
Epoch 4/50
750/750 [==============================] - 0s 70us/step - loss: 0.5964 - accuracy: 0.7107 - val_loss: 0.6325 - val_accuracy: 0.6600
Epoch 5/50
750/750 [==============================] - 0s 73us/step - loss: 0.5840 - accuracy: 0.7133 - val_loss: 0.6238 - val_accuracy: 0.6600
Epoch 6/50
750/750 [==============================] - 0s 76us/step - loss: 0.5737 - accuracy: 0.7133 - val_loss: 0.6129 - val_accuracy: 0.6600
Epoch 7/50
750/750 [==============================] - 0s 73us/step - loss: 0.5646 - accuracy: 0

In [36]:
value_predict = data_copy.iloc[0:2].drop('Risk', axis=1)
model.predict(value_predict)

array([[0.11977863],
       [0.52558374]], dtype=float32)

In [5]:
# Creation of the Classifier
ml_model = KerasClassifier(model=model)